# Intelligent Query Crafting with BigQuery Context: Leveraging Vertex GenAI for Enhanced Business Insights on GCP

In [1]:
import os
GOOGLE_APPLICATION_CREDENTIALS = "/Users/zacharynguyen/Documents/GitHub/2024/Applied-Generative-AI/IAM/zacharynguyen-genai-656c475b142a.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = GOOGLE_APPLICATION_CREDENTIALS

In [2]:
PROJECT_ID='zacharynguyen-genai'
REGION = 'us-central1'
EXPERIMENT = 'covid19-nyt'
SERIES = 'applied-genai'

In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import vertexai.language_models
import vertexai.preview.generative_models
from google.cloud import bigquery

In [4]:
# vertex ai clients
vertexai.init(project = PROJECT_ID, location = REGION)
# bigquery client
bq = bigquery.Client(project = PROJECT_ID)

## GOAL

In [5]:
question = "On the last day of 2022, which state has the most covid-19 confirmed death?"

##  Vertex LLM Setup

In [6]:
# create links to model: embedding api and text generation
textgen_model = vertexai.language_models.TextGenerationModel.from_pretrained('text-bison@002')
codegen_model = vertexai.language_models.CodeGenerationModel.from_pretrained('code-bison@002')
gemini_model = vertexai.preview.generative_models.GenerativeModel("gemini-pro")

In [7]:
textgen_model.predict(f"Write a Google SQL query that answers the following question.\nquestion: {question}")

 ```sql
SELECT state, MAX(deaths) AS max_deaths
FROM `bigquery-public-data.covid19_usafacts.deaths`
WHERE date = '2022-12-31'
GROUP BY state
ORDER BY max_deaths DESC
LIMIT 1;
```

In [8]:
codegen_model.predict(f"Write a Google SQL query that answers the following question.\nquestion: {question}")

```sql
SELECT state, MAX(deaths) AS max_deaths
FROM `bigquery-public-data.covid19_usafacts.deaths`
WHERE date = '2022-12-31'
GROUP BY state
ORDER BY max_deaths DESC
LIMIT 1;
```

In [9]:
print(gemini_model.generate_content(f"Write a Google SQL query that answers the following question.\nquestion: {question}", generation_config = dict(temperature=0)).text)

```sql
SELECT
  state,
  COUNT(*) AS total_deaths
FROM covid19_confirmed_deaths
WHERE
  date = (
    SELECT
      MAX(date)
    FROM covid19_confirmed_deaths
    WHERE
      date LIKE '2022-12%'
  )
GROUP BY
  state
ORDER BY
  total_deaths DESC
LIMIT 1;
```


### The Challenge

While Large Language Models (LLMs) are capable of generating SQL queries, this process encounters notable obstacles:

- The queries often fail to target the appropriate tables.
- Incorrect column names are frequently used, not matching those in the intended tables.
- Essentially, the SQL produced serves more as an initial draft, requiring further refinement by the user to accurately fetch the necessary data in response to their query.

### Achieving Executable SQL Queries via LLM

To address these issues and generate fully operational SQL queries, a methodical strategy was developed through the repeated optimization of text prompts and methodologies tailored to distinct inquiries.

### Retrieve Table Schemas

In [10]:
BQ_PROJECT = 'bigquery-public-data'
BQ_DATASET = 'covid19_nyt'
BQ_TABLES = ['excess_deaths', 'mask_use_by_county', 'us_counties', 'us_states']

In [11]:

query = f"""
    SELECT * EXCEPT(field_path, collation_name, rounding_mode)
    FROM `{BQ_PROJECT}.{BQ_DATASET}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
    WHERE table_name in ({','.join([f'"{table}"' for table in BQ_TABLES])})
"""
print(query)
schema_columns = bq.query(query = query).to_dataframe()


    SELECT * EXCEPT(field_path, collation_name, rounding_mode)
    FROM `bigquery-public-data.covid19_nyt.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
    WHERE table_name in ("excess_deaths","mask_use_by_county","us_counties","us_states")


In [12]:
schema_columns.shape

(29, 6)

In [13]:
schema_columns.head()

,table_catalog,table_schema,table_name,column_name,data_type,description
0,bigquery-public-data,covid19_nyt,excess_deaths,country,STRING,The country reported
1,bigquery-public-data,covid19_nyt,excess_deaths,placename,STRING,The place in the country reported
2,bigquery-public-data,covid19_nyt,excess_deaths,frequency,STRING,"Weekly or monthly, depending on how the data i..."
3,bigquery-public-data,covid19_nyt,excess_deaths,start_date,DATE,The first date included in the period
4,bigquery-public-data,covid19_nyt,excess_deaths,end_date,DATE,The last date included in the period


In [41]:
schema_columns.to_markdown

<bound method DataFrame.to_markdown of            table_catalog table_schema          table_name       column_name  \
0   bigquery-public-data  covid19_nyt       excess_deaths           country   
1   bigquery-public-data  covid19_nyt       excess_deaths         placename   
2   bigquery-public-data  covid19_nyt       excess_deaths         frequency   
3   bigquery-public-data  covid19_nyt       excess_deaths        start_date   
4   bigquery-public-data  covid19_nyt       excess_deaths          end_date   
5   bigquery-public-data  covid19_nyt       excess_deaths              year   
6   bigquery-public-data  covid19_nyt       excess_deaths             month   
7   bigquery-public-data  covid19_nyt       excess_deaths              week   
8   bigquery-public-data  covid19_nyt       excess_deaths            deaths   
9   bigquery-public-data  covid19_nyt       excess_deaths   expected_deaths   
10  bigquery-public-data  covid19_nyt       excess_deaths     excess_deaths   
11  bigquery-

### Code Generation LLM - With Context

In [14]:
print(question)

On the last day of 2022, which state has the most covid-19 confirmed death?


In [15]:
context_prompt = f"""
context (BigQuery Table Schema):
{schema_columns.to_markdown(index = False)}

Write a query for Google BigQuery using fully qualified table names to answer this question:
{question}
"""

#context_query = textgen_model.predict(f"Write a Google SQL query that answers the following question.\nquestion: {question}")
#context_query = codegen_model.predict(context_prompt, max_output_tokens = 500)
context_query = gemini_model.generate_content(context_prompt, generation_config = dict(temperature = 0))

print(context_query.text)

```sql
SELECT
  state_name,
  MAX(deaths) AS max_deaths
FROM `bigquery-public-data.covid19_nyt.us_states`
WHERE
  date = (
    SELECT
      MAX(date)
    FROM `bigquery-public-data.covid19_nyt.us_states`
    WHERE
      date <= '2022-12-31'
  )
GROUP BY
  state_name
ORDER BY
  max_deaths DESC
LIMIT 1;
```


In [16]:
context_query = query = '\n'.join(context_query.text.split('\n')[1:-1])
print(context_query)

SELECT
  state_name,
  MAX(deaths) AS max_deaths
FROM `bigquery-public-data.covid19_nyt.us_states`
WHERE
  date = (
    SELECT
      MAX(date)
    FROM `bigquery-public-data.covid19_nyt.us_states`
    WHERE
      date <= '2022-12-31'
  )
GROUP BY
  state_name
ORDER BY
  max_deaths DESC
LIMIT 1;


In [17]:
dry_run = bq.query(context_query, job_config = bigquery.QueryJobConfig(dry_run = True, use_query_cache = False))

In [18]:
dry_run.errors

In [19]:
dry_run.total_bytes_processed

1674827

In [20]:
context_response = bq.query(context_query).to_dataframe()
context_response

,state_name,max_deaths
0,California,100865


### Answer The Question
Now that a valid context has been retrieved from BigQuery it can be passed to a text generation LLM to answer the user questions.

In [21]:
question_prompt = f"""
context (result from BigQuery query):
{context_response.to_markdown(index = False)}

Answer the following question using the provided context.  Note that the context is a tabular result returned from a BigQuery query.  Do not repeat the question or the context when responding.
{question}
"""

question_response = textgen_model.predict(question_prompt)

print(question_response.text)

 California


### Put It All Together
Ask a new question and try it out:

In [29]:
question = 'Which state have the least confirmed case?'

In [30]:
context_prompt = f"""
context (BigQuery Table Schema):
{schema_columns.to_markdown(index = False)}

Write a query for Google BigQuery using fully qualified table names to answer this question:
{question}
"""

#context_query = textgen_model.predict(f"Write a Google SQL query that answers the following question.\nquestion: {question}")
#context_query = codegen_model.predict(context_prompt, max_output_tokens = 500)
context_query = gemini_model.generate_content(context_prompt, generation_config = dict(temperature = 0))


context_response = bq.query(query = '\n'.join(context_query.text.split('\n')[1:-1])).to_dataframe()

In [31]:
print(context_response.to_markdown(index = False))

| state_name   |
|:-------------|
| Alaska       |


In [32]:
question_prompt = f"""
context (result from BigQuery query):
{context_response.to_markdown(index = False)}

Answer the following question.  Note that the context is a tabular result returned from a BigQuery query.  Do not repeat the question or the context when responding.
{question}
"""

question_response = textgen_model.predict(question_prompt)

print(question_response.text)

 Alaska


### More Complex

In [36]:
question = 'Which county has the highest total number of confirmed deaths of COVID-19?'

In [37]:
context_prompt = f"""
context (BigQuery Table Schema):
{schema_columns.to_markdown(index = False)}

Write a query for Google BigQuery using fully qualified table names to answer this question:
{question}
"""

#context_query = textgen_model.predict(f"Write a Google SQL query that answers the following question.\nquestion: {question}")
#context_query = codegen_model.predict(context_prompt, max_output_tokens = 500, temperature = 0.9)
context_query = gemini_model.generate_content(context_prompt, generation_config = dict(temperature = 0))

context_query = '\n'.join(context_query.text.split('\n')[1:-1])
context_response = bq.query(query = context_query).to_dataframe()

In [38]:

print(context_query)

SELECT
  county,
  SUM(deaths) AS total_deaths
FROM
  `bigquery-public-data.covid19_nyt.us_counties`
GROUP BY
  county
ORDER BY
  total_deaths DESC
LIMIT 1;


In [39]:
context_response

,county,total_deaths
0,New York City,32697668


In [40]:
question_prompt = f"""
Answer the following question.  Note that the context above is a tabular result returned from a BigQuery query specific to this question.  Do not repeat the question or the context when responding.
{question}

Use this data:
{context_response.to_markdown(index = False)}
"""

question_response = gemini_model.generate_content(question_prompt)

print(question_response.text)

New York City
